In [1]:
import warnings

import numpy as np
from sklearn.datasets import load_breast_cancer, load_iris

# Iris

In [2]:
iris = load_iris()

### Implement Naive Bayes classifier

In [3]:
class MyGaussianNB:
    def __init__(self):
        pass

    def fit(self, X, y):
        self.class_labels, self.class_counts = np.unique(y, return_counts=True)
        self.class_priors = np.log(self.class_counts / self.class_counts.sum())

        class_means = []
        class_variance = []
        for label in self.class_labels:
            current_X = X[np.where(y == label)]
            mean = np.mean(current_X, axis=0)
            double_variance = np.var(current_X, axis=0)
            class_means.append(mean)
            class_variance.append(double_variance)

        self.class_means = np.array(class_means)
        self.class_variance = 2 * np.array(class_variance)

    def predict(self, X):

        # for proper broadcasting and optimal numpy we need to extend dimensions
        means = self.class_means[None, :, :]
        variances = self.class_variance[None, :, :]

        with warnings.catch_warnings():
            # suppress 0 division warnings I guess
            warnings.simplefilter("ignore")
            norms = np.log(
                np.exp(-np.square(X[:, None, :] - means) / (variances))
                / np.sqrt(np.pi * variances)
            )

        # norms are a vector of (60,3,4 values) - log_probabilities for each dimension, for each class, fore each X
        probas = norms.sum(axis=-1) + self.class_priors

        # at this point probas is a (X.shape[0],n_classes) vector of summed log proba values for each class
        return self.class_labels[probas.argmax(axis=-1)]

### Check the results of naive bayes classifier

In [4]:
from sklearn.metrics import accuracy_score, f1_score, precision_score
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
import warnings


my_pred = []
sklearn_pred = []
actual_labels = []

for i in range(20):
    X_train, X_test, y_train, y_test = train_test_split(
        iris["data"],
        iris["target"],
        train_size=0.6,
        stratify=iris["target"],
        random_state=i,
    )

    actual_labels.append(y_test)

    clf_mine = MyGaussianNB()
    clf_mine.fit(X_train, y_train)
    my_pred.append(clf_mine.predict(X_test))

    clf_sklearn = GaussianNB()
    clf_sklearn.fit(X_train, y_train)

    sklearn_pred.append(clf_sklearn.predict(X_test))

my_pred = np.concatenate(my_pred)
sklearn_pred = np.concatenate(sklearn_pred)
actual_labels = np.concatenate(actual_labels)

print(f"my acc       : {accuracy_score(my_pred,actual_labels)}")
print(f"my f1        : {f1_score(my_pred,actual_labels,average=None)}")
print(f"my prec      : {precision_score(my_pred,actual_labels,average=None)}")
print()
print(f"sklearn acc  : {accuracy_score(sklearn_pred,actual_labels)}")
print(f"sklearn f1   : {f1_score(sklearn_pred,actual_labels,average=None)}")
print(f"sklearn prec : {precision_score(sklearn_pred,actual_labels,average=None)}")
print()
score_match = (my_pred == sklearn_pred).mean() * 100
print(f"Results of custom and library classifiers match in {score_match}%")

my acc       : 0.96
my f1        : [1.         0.94014963 0.93984962]
my prec      : [1.     0.9425 0.9375]

sklearn acc  : 0.96
sklearn f1   : [1.         0.94014963 0.93984962]
sklearn prec : [1.     0.9425 0.9375]

Results of custom and library classifiers match in 100.0%


The scores returned by my classifier are identical to those returned by scikit-learn GaussianNB

# Breast Cancer

In [5]:
breast_cancer = load_breast_cancer()

In [6]:
X_train, X_test, y_train, y_test = train_test_split(
    breast_cancer["data"], breast_cancer["target"],
    train_size=0.7,
    stratify=breast_cancer["target"],
    random_state=42,
)

### Standardization

In [7]:
clf = MyGaussianNB()
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)

print(f"acc  : {accuracy_score(y_pred,y_test)}")
print(f"f1   : {f1_score(y_pred,y_test)}")
print(f"prec : {precision_score(y_pred,y_test)}")

acc  : 0.935672514619883
f1   : 0.9493087557603687
prec : 0.9626168224299065


In [8]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

clf = Pipeline([("scaler",StandardScaler()),("nb",MyGaussianNB())])

clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)

print(f"acc  : {accuracy_score(y_pred,y_test)}")
print(f"f1   : {f1_score(y_pred,y_test)}")
print(f"prec : {precision_score(y_pred,y_test)}")

acc  : 0.935672514619883
f1   : 0.9493087557603687
prec : 0.9626168224299065


It's a well known fact that naive bayes is not impacted by scaling as the features create independent distributions

### PCA

In [9]:
clf = MyGaussianNB()
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)

accuracy = accuracy_score(y_pred,y_test)
f1 = f1_score(y_pred,y_test)
precision = precision_score(y_pred,y_test)

best_accuracy = "baseline"
best_f1 = "baseline"
best_precision = "baseline"

In [10]:
from sklearn.decomposition import PCA
from sklearn.model_selection import ParameterGrid

# PCA requires standardization
clf = Pipeline([("scaler",StandardScaler()),("pca",PCA()),("nb",MyGaussianNB())])

param_grid = {
    "pca__n_components":[25,20,15,10,5],
}

for grid in ParameterGrid(param_grid):
    clf.set_params(**grid)

    clf.fit(X_train,y_train)
    y_pred = clf.predict(X_test)
    
    accuracy_ = accuracy_score(y_pred,y_test)
    f1_ = f1_score(y_pred,y_test)
    precision_ = precision_score(y_pred,y_test)
    
    if accuracy_ >= accuracy:
        best_accuracy = grid
        accuracy = accuracy_
    
    if f1_ >= f1:
        best_f1 = grid
        f1 = f1_
    
    if precision_ >= precision:
        best_precision = grid
        precision = precision_

print(f"grid for best accuracy  ({accuracy}) : \n {best_accuracy}")
print(f"grid for best f1        ({f1}) : \n {best_f1}")
print(f"grid for best precision ({precision}) : \n {best_precision}")

grid for best accuracy  (0.935672514619883) : 
 {'pca__n_components': 5}
grid for best f1        (0.9502262443438914) : 
 {'pca__n_components': 5}
grid for best precision (0.9813084112149533) : 
 {'pca__n_components': 5}


### Kernel PCA

In [11]:
clf = MyGaussianNB()
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)

accuracy = accuracy_score(y_pred,y_test)
f1 = f1_score(y_pred,y_test)
precision = precision_score(y_pred,y_test)

best_accuracy = "baseline"
best_f1 = "baseline"
best_precision = "baseline"

In [12]:
from sklearn.decomposition import KernelPCA

# PCA requires standardization
clf = Pipeline([("scaler",StandardScaler()),("pca",KernelPCA()),("nb",MyGaussianNB())])

param_grid = {
    "pca__n_components":[25,20,15,10,5],
    "pca__kernel" : ["linear", "poly", "rbf", "sigmoid", "cosine"]
}

for grid in ParameterGrid(param_grid):
    clf.set_params(**grid)

    clf.fit(X_train,y_train)
    y_pred = clf.predict(X_test)
    
    accuracy_ = accuracy_score(y_pred,y_test)
    f1_ = f1_score(y_pred,y_test)
    precision_ = precision_score(y_pred,y_test)
    
    if accuracy_ >= accuracy:
        best_accuracy = grid
        accuracy = accuracy_
    
    if f1_ >= f1:
        best_f1 = grid
        f1 = f1_
    
    if precision_ >= precision:
        best_precision = grid
        precision = precision_

print(f"grid for best accuracy  ({accuracy}) : \n {best_accuracy}")
print(f"grid for best f1        ({f1}) : \n {best_f1}")
print(f"grid for best precision ({precision}) : \n {best_precision}")

grid for best accuracy  (0.9473684210526315) : 
 {'pca__kernel': 'cosine', 'pca__n_components': 10}
grid for best f1        (0.958904109589041) : 
 {'pca__kernel': 'cosine', 'pca__n_components': 10}
grid for best precision (0.9813084112149533) : 
 {'pca__kernel': 'cosine', 'pca__n_components': 10}


We managed to significantly improve the scores using Kernel PCA with cosine kernel and 10 components

### Box-Cox

In [18]:
# TODO BOX-COX

In [ ]:
clf = MyGaussianNB()
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)

accuracy = accuracy_score(y_pred,y_test)
f1 = f1_score(y_pred,y_test)
precision = precision_score(y_pred,y_test)

best_accuracy = "baseline"
best_f1 = "baseline"
best_precision = "baseline"

In [17]:
# from sklearn.preprocessing import PowerTransformer
# 
# clf = Pipeline([("boxcox",PowerTransformer(method="box-cox", standardize=False,alpha)),("nb",MyGaussianNB())])
# 
# for lambda_param in [[0.0], [0.1], [0.2], [0.3], [0.4], [0.5], [0.6], [0.7], [0.8], [0.9], [1.0]]:
#     clf["boxcox"].lambdas_ = lambda_param
# 
#     clf.fit(X_train,y_train)
#     y_pred = clf.predict(X_test)
#     
#     accuracy_ = accuracy_score(y_pred,y_test)
#     f1_ = f1_score(y_pred,y_test)
#     precision_ = precision_score(y_pred,y_test)
#     
#     if accuracy_ >= accuracy:
#         best_accuracy = grid
#         accuracy = accuracy_
#     
#     if f1_ >= f1:
#         best_f1 = grid
#         f1 = f1_
#     
#     if precision_ >= precision:
#         best_precision = grid
#         precision = precision_
# 
# print(f"grid for best accuracy  ({accuracy}) : \n {best_accuracy}")
# print(f"grid for best f1        ({f1}) : \n {best_f1}")
# print(f"grid for best precision ({precision}) : \n {best_precision}")

### Binomial Distribution NB

In [19]:
class MyBinomialNB:
    def __init__(self,alpha=1.0):
        self.alpha = alpha

    def fit(self, X, y):
        self.class_labels, self.class_counts = np.unique(y, return_counts=True)
        self.class_priors = np.log(self.class_counts / self.class_counts.sum())

        class_probs = []
        for label in self.class_labels:
            current_X = X[np.where(y == label)]
            
            # laplace smoothing (x.sum()+1)/(|x|+2) instead of x.sum()/|x|
            theta = (np.sum(current_X, axis=0) + self.alpha) / (current_X.shape[0] + self.alpha*2) 
            class_probs.append(theta)
            
            class_probs.append(np.mean(current_X, axis=0))

        # Thetas
        self.class_probs = np.array(class_probs)

    def predict(self, X):    
        log_prob_1 = np.log(self.class_probs)[None, :, :]
        log_prob_0 = np.log(1 - self.class_probs)[None, :, :]
        
        likelihoods = (X[:, None, :] * log_prob_1 + (1 - X)[:, None, :] * log_prob_0).sum(axis=-1)
        
        probas = likelihoods + self.class_priors

        # Return the class with the highest probability for each sample
        return self.class_labels[probas.argmax(axis=-1)]

In [ ]:
from sklearn.naive_bayes import BernoulliNB
BernoulliNB()